In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

japanize_matplotlib is already installed.
catboost is already installed.
optuna is already installed.
geolib is already installed.
Directory and file for OpenCL vendors have been created.


In [ ]:
import pandas as pd
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

from utils import (ProcessData,
                   trans_chara,
                   fill_missing_values,
                   fill_missing_with_mode,
                   find_nearest_station_main_table,
                   calculate_distance,
                   convert_kanji_to_fullwidth_2,
                   encode_geohash,
                   to_cat_col,
                   TreeExecution,
                   check_dataframe_elements
                   )

TARGET="単位面積あたりの取引価格_log"

In [ ]:
files = glob.glob(
    r"/content/drive/MyDrive/yuto/project_directory/data/competition_data/train/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)
df_copy=df.copy()
check_dataframe_elements(df)

df_test=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/competition_data/test.csv",index_col=0)
df_test_copy=df_test.copy()


# カラム 'A' の欠損値の数を確認
tre_missing_values_count = df['最寄駅：名称'].isnull().sum()
tes_missing_values_count = df_test['最寄駅：名称'].isnull().sum()
print(f"学習データの欠損値の数: {tre_missing_values_count}")
print(f"テストデータの欠損値の数: {tes_missing_values_count}")

<ipython-input-45-9e28a822621a>:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
<ipython-input-45-9e28a822621a>:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


<class 'pandas.core.frame.DataFrame'>
Index: 1009271 entries, 35007328 to 6009259
Data columns (total 28 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   種類            1009271 non-null  object 
 1   価格情報区分        1009271 non-null  object 
 2   地域            0 non-null        float64
 3   市区町村コード       1009271 non-null  int64  
 4   都道府県名         1009271 non-null  object 
 5   市区町村名         1009271 non-null  object 
 6   地区名           1008610 non-null  object 
 7   最寄駅：名称        998620 non-null   object 
 8   最寄駅：距離（分）     964322 non-null   object 
 9   間取り           976570 non-null   object 
 10  面積（㎡）         1009271 non-null  object 
 11  土地の形状         0 non-null        float64
 12  間口            0 non-null        float64
 13  延床面積（㎡）       0 non-null        float64
 14  建築年           979931 non-null   object 
 15  建物の構造         977046 non-null   object 
 16  用途            734705 non-null   object 
 17  今後の利用目的       463475 non-

In [ ]:
df

,種類,価格情報区分,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
35007328,中古マンション等,不動産取引価格情報,NaN,35201,山口県,下関市,秋根西町,新下関,2,３ＬＤＫ,...,NaN,NaN,NaN,商業,80.0,400.0,2012年第2四半期,未改装,NaN,7.146128
35048345,中古マンション等,不動産取引価格情報,NaN,35215,山口県,周南市,五月町,徳山,30分～60分,３ＬＤＫ,...,NaN,NaN,NaN,１種住居,60.0,200.0,2021年第2四半期,未改装,NaN,6.924279
35003676,中古マンション等,不動産取引価格情報,NaN,35201,山口県,下関市,山の田東町,幡生,14,３ＬＤＫ,...,NaN,NaN,NaN,１種住居,60.0,200.0,2021年第2四半期,未改装,NaN,7.204120
35053427,中古マンション等,成約価格情報,NaN,35216,山口県,山陽小野田市,大字山川,厚狭,3,４ＬＤＫ,...,NaN,NaN,NaN,NaN,NaN,NaN,2022年第2四半期,NaN,NaN,7.204120
35005127,中古マンション等,不動産取引価格情報,NaN,35201,山口県,下関市,一の宮町,新下関,2,４ＬＤＫ,...,NaN,NaN,NaN,商業,80.0,400.0,2010年第2四半期,未改装,NaN,7.146128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6018727,中古マンション等,不動産取引価格情報,NaN,6204,山形県,酒田市,幸町,酒田,3,３ＬＤＫ,...,NaN,NaN,NaN,商業,80.0,500.0,2023年第1四半期,未改装,NaN,7.579784
6001936,中古マンション等,不動産取引価格情報,NaN,6201,山形県,山形市,十日町,山形,12,１Ｋ,...,NaN,NaN,NaN,商業,80.0,600.0,2018年第4四半期,改装済み,NaN,6.477121
6008180,中古マンション等,不動産取引価格情報,NaN,6201,山形県,山形市,小荷駄町,山形,21,４ＬＤＫ,...,NaN,NaN,NaN,２中住専,60.0,200.0,2007年第4四半期,未改装,NaN,7.431364


In [ ]:
# データの前処理 ＆ 特徴量エンジニアリング
data_tree=ProcessData(df,df_test)
df_train_tree,df_test_tree,df_all_tree,df_geo,cat_cols=data_tree.return_data_test()

MultiIndex([('train', 35007328),
            ('train', 35048345),
            ('train', 35003676),
            ('train', 35053427),
            ('train', 35005127),
            ('train', 35037465),
            ('train', 35001173),
            ('train', 35049701),
            ('train', 35001661),
            ('train', 35012625),
            ...
            ( 'test', 47036042),
            ( 'test', 47036043),
            ( 'test', 47036044),
            ( 'test', 47036616),
            ( 'test', 47036619),
            ( 'test', 47036620),
            ( 'test', 47037115),
            ( 'test', 47037850),
            ( 'test', 47037855),
            ( 'test', 47037856)],
           names=[None, 'ID'], length=1063673)


In [ ]:
# マルチインデックスを解除
df_all_tree_reset = df_all_tree.reset_index()

# カラム名を適切に設定
df_all_tree_reset.columns = ['dataset', 'ID'] + list(df_all_tree_reset.columns[2:])

df_all_tree=df_all_tree_reset.copy()
del df_all_tree_reset

In [ ]:
#check_dataframe_elements(df_train_tree)

「国土地理院API」「駅データ」読み込み

In [ ]:
# 駅データ読み込み
df_station=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/external_data/station20240426free.csv")
#df_station02=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/external_data/station20240328free.csv")
df_pref=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/external_data/pref.csv")#駅データの都道府県名

df_station=pd.merge(df_station,df_pref,how="left",left_on="pref_cd",right_on="pref_cd")


#ジオコード作成データを取得
df_all_address=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/all_address.csv")
df_address=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/address.csv")

#df_all_address=df_all_address.rename(columns={"x":"x_api","y":"y_api"})
#df_address=df_address.rename(columns={"x":"x_api","y":"y_api"})

df_all_address=df_all_address.rename(columns={"x":"x_api","y":"y_api","最寄駅_名称":"最寄駅：名称","最寄駅_距離_分":"最寄駅：距離（分）"})
df_address=df_address.rename(columns={"x":"x_api","y":"y_api","最寄駅_名称":"最寄駅：名称","最寄駅_距離_分":"最寄駅：距離（分）"})

#この時点でのaddressは後にメインテーブルと結合時に使用する
df_address['住所']=df_address['address']

国土地理院APIから返ってきた住所「title」とAPIに送信した住所「address」の名寄せを行い\
取得住所の信頼性が無い or 地区名が欠損値を表す\
「住所不一致_Flag」を作成する

In [ ]:
df_address['title']=(
    df_address['title']
    .fillna("unknown")
    .str.replace('ケ', 'ヶ')
    .str.replace(r"\（.*\）","",regex=True)
    .str.replace(r"\〈.*\〉","",regex=True)
    .apply(trans_chara)
    )

df_address["address"]=(
    df_address['address']
    .fillna("unknown")
    .str.replace('ケ', 'ヶ')
    .str.replace(r"\（.*\）","",regex=True)
    .str.replace(r"\〈.*\〉","",regex=True)
    .apply(convert_kanji_to_fullwidth_2)
    )

df_address["title"]=df_address["title"].apply(convert_kanji_to_fullwidth_2)

# 新しいカラムを作成
#df_address['住所不一致_Flag'] = df_address.apply(lambda row: 0 if row['address'] in row['title'] else 1, axis=1)
df_address['住所不一致_Flag'] = df_address.apply(lambda row: 1 if pd.isna(row['地区名']) or row['address'] not in row['title'] else 0, axis=1)

# 新しいカラムを作成
#df_address['住所不一致_Flag'] = df_address.apply(lambda row: 0 if re.search(re.escape(convert_kanji_to_fullwidth_2(row['title'])), row['address']) else 1, axis=1)
#df_address['住所不一致_Flag'] = df_address.apply(lambda row: 0 if row['address'], re.search(re.escape(convert_kanji_to_fullwidth_2(row['title'])) else 1, axis=1)

駅名の表記ゆれ修正

In [ ]:
df_station["最寄駅：名称"]=(
    df_station["station_name"]
    .fillna("unknown")
    .str.replace("ケ","ヶ")
    .str.replace(r"\（.*\）","",regex=True)
    .str.replace(r"\〈.*\〉","",regex=True)
    .apply(trans_chara)
)
df_station=df_station.drop_duplicates(subset=["最寄駅：名称"])

In [ ]:
df=df_all_tree.copy()

In [ ]:
# 地区名が欠損値か否かを表すカラムを作成
df["住所_地区名欠損フラグ"]=df["地区名"].isna().astype(int)
# 住所を作成
df['住所'] = df['都道府県名'] + df['市区町村名'] + df['地区名'].fillna('')
# 徒歩時間を10分単位で丸める
df['徒歩時間_10分単位'] = (df['最寄駅：距離（分）'] // 10) * 10

国土地理院APIの情報を住所で結合\
メインテーブルに対して結合

In [ ]:
df = pd.merge(df, df_address[['住所', 'address', 'title', 'x_api', 'y_api','住所不一致_Flag']], on='住所', how='left')

# 外部データに名前寄せする
df['最寄駅：名称'] = (
    df['最寄駅：名称']
    .fillna('unknown')
    .str.replace(r'\(.*\)', '', regex=True)
    .str.replace('ケ', 'ヶ')
    .str.replace('\u3000', ' ') # トヨタモビリティ富山 Gスクエア五福前に含まれている
    .str.replace('祗園橋', '祇園橋')
    .apply(trans_chara)
)

駅データの経度緯度の情報を駅名で結合
メインテーブルに対して結合

In [ ]:
#この時点では左のデータフレームに存在する駅名のみ反映される(後々補間あり)
df=pd.merge(df,df_station[['最寄駅：名称','lon','lat']],on=['最寄駅：名称'], how='left')
# 最寄駅：名称でユニークな行を残す
df_unique = df.drop_duplicates(subset='最寄駅：名称')

In [ ]:
df

,dataset,ID,価格情報区分,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,住所_地区名欠損フラグ,住所,徒歩時間_10分単位,address,title,x_api,y_api,住所不一致_Flag,lon,lat
0,train,35007328,不動産取引価格情報,山口県,下関市,秋根西町,新下関,2.0,３ＬＤＫ,75.0,...,0,山口県下関市秋根西町,0.0,山口県下関市秋根西町,山口県下関市秋根西町,130.947525,34.009121,0,130.948021,34.005651
1,train,35048345,不動産取引価格情報,山口県,周南市,五月町,徳山,45.0,３ＬＤＫ,65.0,...,0,山口県周南市五月町,40.0,山口県周南市５月町,山口県周南市５月町,131.829285,34.038544,0,131.802900,34.051555
2,train,35003676,不動産取引価格情報,山口県,下関市,山の田東町,幡生,14.0,３ＬＤＫ,70.0,...,0,山口県下関市山の田東町,10.0,山口県下関市山の田東町,山口県下関市山の田東町,130.928864,33.985626,0,130.927340,33.981592
3,train,35053427,成約価格情報,山口県,山陽小野田市,大字山川,厚狭,3.0,４ＬＤＫ,85.0,...,0,山口県山陽小野田市大字山川,0.0,山口県山陽小野田市山川,山口県山陽小野田市山川,131.151978,34.069763,0,131.159965,34.053361
4,train,35005127,不動産取引価格情報,山口県,下関市,一の宮町,新下関,2.0,４ＬＤＫ,75.0,...,0,山口県下関市一の宮町,0.0,山口県下関市１の宮町,山口県下関市１の宮町,130.946640,34.003754,0,130.948021,34.005651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063668,test,47036620,成約価格情報,沖縄県,中頭郡北中城村,字ライカム,ｕｎｋｎｏｗｎ,NaN,２ＬＤＫ,65.0,...,0,沖縄県中頭郡北中城村字ライカム,NaN,沖縄県中頭郡北中城村ライカム,沖縄県中頭郡北中城村ライカム,127.796249,26.312994,0,NaN,NaN
1063669,test,47037115,成約価格情報,沖縄県,中頭郡中城村,字南上原,ｕｎｋｎｏｗｎ,NaN,２ＬＤＫ,70.0,...,0,沖縄県中頭郡中城村字南上原,NaN,沖縄県中頭郡中城村南上原,沖縄県中頭郡中城村南上原,127.771294,26.250996,0,NaN,NaN
1063670,test,47037850,成約価格情報,沖縄県,中頭郡西原町,字翁長,ｕｎｋｎｏｗｎ,NaN,３ＬＤＫ,65.0,...,0,沖縄県中頭郡西原町字翁長,NaN,沖縄県中頭郡西原町翁長,沖縄県中頭郡西原町翁長,127.752129,26.233952,0,NaN,NaN
1063671,test,47037855,成約価格情報,沖縄県,中頭郡西原町,上原,ｕｎｋｎｏｗｎ,NaN,２ＬＤＫ,75.0,...,0,沖縄県中頭郡西原町上原,NaN,沖縄県中頭郡西原町上原,沖縄県中頭郡西原町上原,127.762131,26.241070,0,NaN,NaN


In [ ]:
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名',
       #'住所_地区名欠損フラグ',
       '住所不一致_Flag',
       '住所',
       #'address',
       #'title',
       '最寄駅：名称','最寄駅：距離（分）',
       '徒歩時間_10分単位','x_api', 'y_api',  'lon', 'lat','間取り',
       '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画',
       '建ぺい率（％）', '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log','単位面積あたりの取引価格_log',
       '建築西暦年', '築年数', 'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ']]

In [ ]:
check_dataframe_elements(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063673 entries, 0 to 1063672
Data columns (total 37 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   dataset            1063673 non-null  object 
 1   ID                 1063673 non-null  int64  
 2   価格情報区分             1063673 non-null  object 
 3   都道府県名              1063673 non-null  object 
 4   市区町村名              1063673 non-null  object 
 5   地区名                1063012 non-null  object 
 6   住所不一致_Flag         1063673 non-null  int64  
 7   住所                 1063673 non-null  object 
 8   最寄駅：名称             1063673 non-null  object 
 9   最寄駅：距離（分）          1014618 non-null  float64
 10  徒歩時間_10分単位         1014618 non-null  float64
 11  x_api              1061959 non-null  float64
 12  y_api              1061959 non-null  float64
 13  lon                1051220 non-null  float64
 14  lat                1051220 non-null  float64
 15  間取り                1029726 non-n

In [ ]:
def imawatukawanai(df):
    df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名',
          #'住所_地区名欠損フラグ',
          '住所不一致_Flag',
          '住所',
          #'address',
          #'title',
          '最寄駅：名称','最寄駅：距離（分）',
          '徒歩時間_10分単位','x_api', 'y_api',  'lon', 'lat','間取り',
          '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画',
          '建ぺい率（％）', '容積率（％）', '取引時点', '改装', '取引の事情等', '単位面積あたりの取引価格_log',
          '建築西暦年', '築年数', 'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
          '面積（㎡）容積率（％）_combi', '旧耐震フラグ']]

In [ ]:
check_dataframe_elements(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063673 entries, 0 to 1063672
Data columns (total 37 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   dataset            1063673 non-null  object 
 1   ID                 1063673 non-null  int64  
 2   価格情報区分             1063673 non-null  object 
 3   都道府県名              1063673 non-null  object 
 4   市区町村名              1063673 non-null  object 
 5   地区名                1063012 non-null  object 
 6   住所不一致_Flag         1063673 non-null  int64  
 7   住所                 1063673 non-null  object 
 8   最寄駅：名称             1063673 non-null  object 
 9   最寄駅：距離（分）          1014618 non-null  float64
 10  徒歩時間_10分単位         1014618 non-null  float64
 11  x_api              1061959 non-null  float64
 12  y_api              1061959 non-null  float64
 13  lon                1051220 non-null  float64
 14  lat                1051220 non-null  float64
 15  間取り                1029726 non-n

経度緯度補間\
住所不一致_Flagが1の場合に住所の経度緯度を駅の経度緯度で補間

In [ ]:
# 住所不一致_Flagが1の場合、x_apiをlonで補間
df['x_api'] = np.where(df['住所不一致_Flag'] == 1, df['lon'], df['x_api'])
# 住所不一致_Flagが1の場合、y_apiをlatで補間
df['y_api'] = np.where(df['住所不一致_Flag'] == 1, df['lat'], df['y_api'])

<ipython-input-62-dcd62821c04a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x_api'] = np.where(df['住所不一致_Flag'] == 1, df['lon'], df['x_api'])
<ipython-input-62-dcd62821c04a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y_api'] = np.where(df['住所不一致_Flag'] == 1, df['lat'], df['y_api'])


NameError: name 'abc' is not defined

地区名補間\
「県名」「駅名」「駅からの徒歩時間を10分単位で丸めたもの」をキーに欠損しているものを補完

In [ ]:
# 欠損値を補完
key_columns = ['都道府県名', '最寄駅：名称', '徒歩時間_10分単位']
target_column = '地区名'
df = fill_missing_values(df, key_columns, target_column)

駅名補間\
住所の最頻値で駅名を補間

In [ ]:
#　住所と最寄駅：名称の最頻値で欠損値を補完
df = fill_missing_with_mode(df, '最寄駅：名称', '住所')

駅名補間\
住所経度緯度から一番近い駅で駅名を補間

In [ ]:
# 欠損値を補完
df['最寄駅：名称'] = df.apply(find_nearest_station_main_table, axis=1, df=df_unique)

#df.to_csv(r"/content/drive/MyDrive/Colab Notebooks/中古マンション価格予測_2024年夏/作成データ/df_all_tree1_main_table.csv")

#df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/df_all_tree1_main_table.csv",index_col=0)

駅からの距離を算出する

In [ ]:
df["最寄駅までの距離"]=df.apply(calculate_distance,axis=1)
#df.to_csv(r"/content/drive/MyDrive/Colab Notebooks/中古マンション価格予測_2024年夏/作成データ/df_all_tree2_main_table.csv")
#df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/df_all_tree2_main_table.csv",index_col=0)

In [ ]:
# 各精度のGeohashをデータフレームに追加
for precision in range(2, 8):
    df[f'geohash_{precision}'] = df.apply(encode_geohash, axis=1, precision=precision)

#df.to_csv(r"C:\Users\yuto2\OneDrive\ドキュメント\Pythonフォルダ\Nishika\中古マンション価格予測_2024夏の部\作成データ\df_all_tree3_main_table.csv")
#df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/df_all_tree3_main_table.csv",index_col=0)

In [ ]:
# 集計特徴量の作成
# Count Encoding
df['県_市_count'] = df.groupby(['都道府県名', '市区町村名'])['都道府県名'].transform('count')
df['建築年_count'] = df.groupby('建築年')['建築年'].transform('count')

# 駅数や路線数の集計（例として駅数を使用）
for precision in range(2, 8):
    df[f'県_市_geohash_{precision}_駅数'] = df.groupby(['都道府県名', '市区町村名', f'geohash_{precision}'])['最寄駅：名称'].transform('count')

# 駅徒歩時間平均値や建築年平均値の集計
for precision in range(2, 8):
    df[f'県_市_geohash_{precision}_駅徒歩時間平均'] = df.groupby(['都道府県名', '市区町村名', f'geohash_{precision}'])['最寄駅：距離（分）'].transform('mean')
    df[f'県_市_geohash_{precision}_建築年平均'] = df.groupby(['都道府県名', '市区町村名', f'geohash_{precision}'])['建築西暦年'].transform('mean')

In [ ]:
#df.to_csv(r"/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv")

In [ ]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

# インデックスを読み込み
train_index = pd.read_csv('/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/train_index.csv', header=None, squeeze=True)
test_index = pd.read_csv('/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/test_index.csv', header=None, squeeze=True)

# インデックスを設定して順序を再現
df_train = df_train.loc[train_index]
df_test = df_test.loc[test_index]

print("Train DataFrame:")
print(df_train)
print("\nTest DataFrame:")
print(df_test)

In [ ]:
##tree_class=TreeExecution(df_train,y_name=TARGET,cat_cols=cat_cols)
#model_lgbm=tree_class.lightgbm_simple()
#fitted_models_lgbm,oof_df=tree_class.lightgbm()
##fitted_models_lgbm,mae_score,oof_df=tree_class.lightgbm()
#model_lgbm,mae_score=tree_class.lightgbm_optuna_holdout()
#fitted_models_cat,mae_score=tree_class.catboost_optuna()
#model_cat,mae_score=tree_class.catboost_optuna_holdout()

#fitted_models_lgbm,oof_pred_lgbm=tree_class.lightgbm()
#fitted_models_cat,oof_pred_cat=tree_class.catboost()

In [ ]:
#oof_df.to_csv(r"/content/drive/MyDrive/Colab Notebooks/中古マンション価格予測_2024年夏/oof_df/oof_lgbm_df_sub.csv")
##m=mean_absolute_error(oof_df["target"],oof_df["pred_lgbm"])
##print(m)